# Deputado Fedral para Qualquer estado - 2018

Creado em: 14/06/2022

Objetivo: Criar csv com votos
+ Com exceçâo de Bahia e de São paulo, não tem votos por ciade, apenas o total de votos (pois demora muito)

Dados do TSE + Voto da Gazeta do Povo

JUNHO DE 2022 
+ Infelsimente a pagina da gazeta com os sitse pessoasi nao funciona
+ Infelismente os dados do TSE nâo funcionam por webScraping

https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/municipios-bahia/deputado-federal-candidato-pastor-sargento-isidorio-7000/

# Import Libs

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os 

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.options.display.float_format = '{:.2f}'.format

# Snippets

In [2]:
def to_df_from_list_of_objs_sorted(list_of_obj):
    """
        Converte de:
    [{'id': 5, 'nome': 'rafael'},  {'id': 6, 'nome': 'marta'}]
        Para DataFrame
    OBS: Esse processo é mais complxo pois tem que ser feito
    """
    variables = list_of_obj[0].keys()

    def aux_obj_to_list_sorted(obj, features):
        alist = []
        for feat in features:
            alist.append(obj[feat])
        return alist

    return pd.DataFrame(
                list( map( lambda x: aux_obj_to_list_sorted(x, variables), list_of_obj)),
                columns = variables)

In [3]:
def convert_long_comment_in_string(long_txt):
    """
    Converte uma string grande em uma lista
    ideal para lsitar as colunas que queremos do dataset apartir deo uso de 3 aspas
    """
    return list(map(lambda x: x.strip(), long_txt.split('\n')[1:-1] ))

In [19]:
def request_get(url, lib_parser = 'html.parser'):
    """
    lib_paeser : 'html.parser' // 
    return BeautifulSoupPage
    """
    response = requests.get(url)
    if(response.status_code != 200):
        print(url, 'Erro Code')
        raise Exception("Request deu Erro")
        quit()
    return BeautifulSoup(response.text, lib_parser)

# Buscar dados do TSE

In [20]:
list_cols_candidato = """
SQ_CANDIDATO
DS_CARGO
NM_CANDIDATO
NR_CANDIDATO
NM_URNA_CANDIDATO
DS_SITUACAO_CANDIDATURA
DS_DETALHE_SITUACAO_CAND
NR_PARTIDO
SG_PARTIDO
NM_PARTIDO
NM_MUNICIPIO_NASCIMENTO
DT_NASCIMENTO
DS_GENERO
DS_GRAU_INSTRUCAO
DS_ESTADO_CIVIL
DS_COR_RACA
DS_OCUPACAO
ST_REELEICAO
DS_SIT_TOT_TURNO
DS_SITUACAO_CANDIDATO_URNA
"""

list_cols_despesas_contratadas = """
DS_CARGO
SQ_CANDIDATO
NR_CANDIDATO
VR_DESPESA_CONTRATADA
"""

list_cols_receitas = """
DS_CARGO
SQ_CANDIDATO
NR_CANDIDATO
VR_RECEITA
"""
    
replace_tse_cols = {
    'SQ_CANDIDATO': 'id_tse',
    'DS_CARGO_x': 'cargo',
    'NM_CANDIDATO': 'nome_completo',
    'NR_CANDIDATO_x': 'numero_candidato',
    'NM_URNA_CANDIDATO': 'nome_urna',
    'DS_SITUACAO_CANDIDATURA': 'situacao_candidatura',
    'DS_DETALHE_SITUACAO_CAND': 'detalhe_situacao_candidatura',
    'NM_PARTIDO': 'nome_partido',
    'NR_PARTIDO': 'numero_partido',
    'SG_PARTIDO': 'sigla_partido',
    'NM_MUNICIPIO_NASCIMENTO': 'municipio_nascimento',
    'DT_NASCIMENTO': 'data_nascimento',
    'DS_GENERO': 'genero',
    'DS_GRAU_INSTRUCAO': 'grau_instrucao',
    'DS_ESTADO_CIVIL': 'estado_civil',
    'DS_COR_RACA': 'cor_raca',
    'DS_OCUPACAO': 'ocupacao',
    'ST_REELEICAO': 'reeleicao',
    'DS_SIT_TOT_TURNO': 'venceu',
    'DS_SITUACAO_CANDIDATO_URNA': 'situacao_candidatura_urna',
    'VR_BEM_CANDIDATO': 'bens',
    'VR_RECEITA': 'receita',
    'VR_DESPESA_CONTRATADA': 'despesa_contratada',
}



In [21]:
files_templates = {
    'df_bens': './../dataset/tse_csv/bem_candidato_2018_XX.csv',
    'df_candidato': './../dataset/tse_csv/consulta_cand_2018_XX.csv',
    'df_despesas_contratadas': './../dataset/tse_csv/despesas_contratadas_candidatos_2018_XX.csv',
    'df_receitas': './../dataset/tse_csv/receitas_candidatos_2018_XX.csv',
}

In [22]:
def replace_float_comma(x):
    return float(x.replace(',','.')) if not pd.isna(x) else x

In [23]:
# Precisa de 4 arquivos do TSE mais o mapemaneto do estado na variavel 'estados'
# no final - so usa o nome em lower
estados = {
    # NORTE
    'AC': ['ac', 'acre'],
    'AM': ['am', 'amazonas'],
    'AP': ['ap', 'amapa'],
    'PA': ['pa', 'para'],
    'RO': ['ro', 'rondonia'],
    'RR': ['rr', 'roraima'],
    'TO': ['to', 'tocantins'],
    # NORDESTE
    'BA': ['ba', 'bahia'],
    'SE': ['se', 'sergipe'],
    'AL': ['al', 'alagoas'],
    'PE': ['pe', 'pernambuco'],
    'PB': ['pb', 'paraiba'],
    'RN': ['rn', 'rio-grande-do-norte'],
    'CE': ['ce', 'ceara'],
    'PI': ['pi', 'piaui'],
    'MA': ['ma', 'maranhao'],
    # CENTRO OESTE
    'DF': ['df', 'distrito-federal'],
    'MS': ['ms', 'mato-grosso-do-sul'],
    'MT': ['mt', 'mato-grosso'],
    'GO': ['go', 'goias'],
    # SUDESTE
    'SP': ['sp', 'sao-paulo'],
    'RJ': ['rj', 'rio-de-janeiro'],
    'MG': ['mg', 'minas-gerais'], 
    'ES': ['es', 'espirito-santo'],
    # SUL
    'PR': ['pr', 'parana'],
    'SC': ['sc', 'santa-catarina'],
    'RS': ['rs', 'rio-grande-do-sul']
    }
index_sigla, index_name = 0, 1

for estado, value in estados.items():
    print('===============', estado, '===============')
    # df_bens
    df_bens = pd.read_csv(
        files_templates['df_bens'].replace('XX',estado), encoding='latin1', sep=';')
    df_bens['VR_BEM_CANDIDATO'] = df_bens['VR_BEM_CANDIDATO'].apply(replace_float_comma)
    df_bens = df_bens.groupby('SQ_CANDIDATO').sum().reset_index()[['SQ_CANDIDATO','VR_BEM_CANDIDATO']]
    # df_candidato
    df_candidato = pd.read_csv(
        files_templates['df_candidato'].replace('XX', estado), encoding='latin1', sep=';')
    df_candidato = df_candidato[convert_long_comment_in_string(list_cols_candidato)]
    # df_despesas_contratadas
    df_despesas_contratadas = pd.read_csv(
        files_templates['df_despesas_contratadas'].replace('XX', estado), encoding='latin1', sep=';')
    df_despesas_contratadas['VR_DESPESA_CONTRATADA'] = df_despesas_contratadas['VR_DESPESA_CONTRATADA'].apply(replace_float_comma)
    df_despesas_contratadas = df_despesas_contratadas[convert_long_comment_in_string(list_cols_despesas_contratadas)]
    df_despesas_contratadas = df_despesas_contratadas.groupby('SQ_CANDIDATO').sum().reset_index()
    # df_receitas
    df_receitas = pd.read_csv(
        files_templates['df_receitas'].replace('XX',estado), encoding='latin1', sep=';')
    df_receitas['VR_RECEITA'] = df_receitas['VR_RECEITA'].apply(replace_float_comma)
    df_receitas = df_receitas[convert_long_comment_in_string(list_cols_receitas)]
    df_receitas = df_receitas.groupby(df_receitas.columns.tolist()[:-1]).sum().reset_index()
    # df_tse
    df_tse = df_candidato.merge(df_bens, how='left', left_on='SQ_CANDIDATO', right_on='SQ_CANDIDATO')
    df_tse = df_tse.merge(df_receitas, how='left', left_on='SQ_CANDIDATO', right_on='SQ_CANDIDATO')
    df_tse = df_tse.merge(df_despesas_contratadas, how='left', left_on='SQ_CANDIDATO', right_on='SQ_CANDIDATO')
    df_tse = df_tse.rename(replace_tse_cols, axis='columns')
    df_tse = df_tse.drop(['NR_CANDIDATO', 'NR_CANDIDATO_y', 'DS_CARGO_y'], axis='columns')
    df_tse = df_tse.replace({'#NULO#': np.NaN, -1: np.NaN})
    
    
    deputado_info_url = 'https://especiais.gazetadopovo.com.br/eleicoes/2018/resultados/eleitos-deputado-federal-XXX-quem-ganhou/'
    deputado_info_url = deputado_info_url.replace('XXX', value[index_sigla])
    soup_gazeta_all_deputado = request_get(deputado_info_url)
    list_tr = soup_gazeta_all_deputado.find('tbody').find_all('tr')
    
    deputado_info_urls = []
    deputado_federal_direct_info = []

    for el in list_tr:
        deputado_federal_info = {}
        tds = el.find_all('td')
        deputado_info_urls.append(tds[0].find('a')['href'])
        deputado_federal_info['number'] = tds[0].find('a')['href'].split('-')[-1].replace('/','')
        deputado_federal_info['name'] =  tds[0].find('h2').get_text().strip().replace(' Eleito','')
        deputado_federal_info['img_url'] =  tds[0].find('img')['data-lazy-src']
        spans = tds[1].find_all('span')
        deputado_federal_info['porcentagem'] = spans[0].get_text().strip()
        deputado_federal_info['votos'] = spans[1].get_text().split(' ')[0].replace('.','')
        deputado_federal_direct_info.append(deputado_federal_info)
        
    # create dataframe votes
    df_deputado_votos = to_df_from_list_of_objs_sorted(deputado_federal_direct_info)
    display(df_deputado_votos.head(3))
    # save
    file_with_votes = './../dataset/generated/deputado_federal_XXX_2018_votes.csv'.replace('XXX', estado)
    df_deputado_votos.to_csv(file_with_votes, index=False, sep=';')
    # load
    df_deputado_votos_re = pd.read_csv(file_with_votes, sep=';')
    df_deputado_votos_re = df_deputado_votos_re[['number', 'porcentagem', 'votos']]
    df_tse = df_tse.merge(df_deputado_votos_re, how='left', left_on='numero_candidato', right_on='number')
    # pre-processing : to federal deputy
    df_tse = pre_processing_fix_some_bugs(base_pre_processing_df_deputado_federal(df_tse))
    
    # save tse
    file_tse_output = './../dataset/generated/deputado_federal_eleicoes_XXX_2018.csv'.replace('XXX', estado)
    df_tse.to_csv(file_tse_output, index=False, sep=';')

=============== AC ===============


,number,name,img_url,porcentagem,votos
0,4545,Mara Rocha,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"9,42%",40047
1,1515,Jéssica Sales,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"6,76%",28717
2,2525,Alan Rick,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"5,24%",22263


=============== AM ===============


,number,name,img_url,porcentagem,votos
0,1313,José Ricardo,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"11,19%",197270
1,1717,Delegado Pablo,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"8,60%",151649
2,1111,Atila Lins,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"6,73%",118700


=============== AP ===============


...

,number,name,img_url,porcentagem,votos
0,5590,Sargento Fahur,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"5,49%",314963
1,1777,Felipe Francischini,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"4,21%",241537
2,1313,Gleisi Lula,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"3,71%",212513


=============== SC ===============


,number,name,img_url,porcentagem,votos
0,1010,Hélio Costa,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"5,05%",179307
1,1717,Daniel Freitas,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"4,02%",142571
2,1350,Professor Pedro Uczai,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"3,25%",115232


=============== RS ===============


,number,name,img_url,porcentagem,votos
0,3030,Marcel van Hattem,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"5,99%",349855
1,2522,Onyx Lorenzoni,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"3,14%",183518
2,2222,Giovani Cherini,https://cdn-eleicoes.gazetadopovo.com.br/fotos...,"2,60%",151719


In [25]:
df_tse.head(1)

,id_tse,cargo,nome_completo,numero_candidato,nome_urna,situacao_candidatura,detalhe_situacao_candidatura,numero_partido,sigla_partido,nome_partido,municipio_nascimento,data_nascimento,genero,grau_instrucao,estado_civil,cor_raca,ocupacao,reeleicao,venceu,situacao_candidatura_urna,bens,receita,despesa_contratada,porcentagem,votos
4,210000609476,DEPUTADO FEDERAL,MARLI ALVES DOS SANTOS TEIXEIRA,6511,MARLI TEIXEIRA EDUCADORA,APTO,DEFERIDO,65,PC do B,PARTIDO COMUNISTA DO BRASIL,OSÓRIO,20/06/1960,FEMININO,ENSINO MÉDIO COMPLETO,CASADO(A),BRANCA,OUTROS,N,NÃO ELEITO,DEFERIDO,42883.00,2543.50,992.68,"0,01%",458.00


In [13]:
def base_pre_processing_df_deputado_federal(df):
    df_tse = df
    # DEPUTADO FEDERAL
    df_tse = df_tse[ df_tse['cargo'] == 'DEPUTADO FEDERAL']

    # CANDIDATOS QUE PARTICIPARAM DA ELEICAO
#     df_tse = df_tse[ df_tse['situacao_candidatura'] == 'APTO']
#     df_tse = df_tse[ df_tse['detalhe_situacao_candidatura'].isin(['DEFERIDO','DEFERIDO COM RECURSO']) ]
#     df_tse = df_tse[ df_tse['situacao_candidatura_urna'].isin(['DEFERIDO','DEFERIDO COM RECURSO']) ]

    # REPLACE ELEITOS E NÃO ELEITOS
    if('cargo_disputado' in df_tse.columns.tolist()):
#         df_tse['cargo_disputado'] = df_tse['cargo_disputado'].replace(
#             { x: 'ELEITO' for x in ['ELEITO POR QP','ELEITO','ELEITO POR MÉDIA'] }
#         )
#         df_tse['cargo_disputado'] = df_tse['cargo_disputado'].replace({'SUPLENTE': 'NÃO ELEITO'})
        df_tse = df_tse.rename({'cargo_disputado': 'venceu'}, axis='columns')
    # VENCEU
#     if('venceu' in df_tse.columns.tolist()):
#         df_tse['venceu'] = df_tse['venceu'].replace(
#             { x: 'ELEITO' for x in ['ELEITO POR QP','ELEITO','ELEITO POR MÉDIA'] }
#         )
#         df_tse['venceu'] = df_tse['venceu'].replace({'SUPLENTE': 'NÃO ELEITO'})
#         df_tse = df_tse.rename({'cargo_disputado': 'venceu'}, axis='columns')
    return df_tse

def pre_processing_fix_some_bugs(df):
    cols = df.columns.tolist()
    dict1, dict2 = {'votos': 'yy', 'porcentagem': 'xx'}, {'yy': 'votos', 'xx': 'porcentagem'}
    df = df.rename(columns=dict1).rename(columns=dict2)
    if('votes' in cols):
        df = df.rename(columns={'votes': 'votos'})
    if('number' in cols):
        df = df.drop(columns=['number'])
    return df

In [18]:
pre_processing_fix_some_bugs(base_pre_processing_df_deputado_federal(df_tse)).head(1)

,id_tse,cargo,nome_completo,numero_candidato,nome_urna,situacao_candidatura,detalhe_situacao_candidatura,numero_partido,sigla_partido,nome_partido,municipio_nascimento,data_nascimento,genero,grau_instrucao,estado_civil,cor_raca,ocupacao,reeleicao,venceu,situacao_candidatura_urna,bens,receita_campanha,despesa_contratada_campanha,porcentagem,votos
4,210000609476,DEPUTADO FEDERAL,MARLI ALVES DOS SANTOS TEIXEIRA,6511,MARLI TEIXEIRA EDUCADORA,APTO,DEFERIDO,65,PC do B,PARTIDO COMUNISTA DO BRASIL,OSÓRIO,20/06/1960,FEMININO,ENSINO MÉDIO COMPLETO,CASADO(A),BRANCA,OUTROS,N,NÃO ELEITO,DEFERIDO,42883.00,2543.50,992.68,"0,01%",458.00
